<a href="https://colab.research.google.com/github/Aine449/Projects-Udacity/blob/main/Pre_trained_Image_Classifier_to_Identify_Dog_Breeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Set up the necessary libraries

In [ ]:
!pip install numpy pandas opencv-python tensorflow torch torchvision


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

Manually add the images

In [ ]:
import os
manual_images = ['/content/pet 1.jpeg', '/content/pet 2.jpeg','/content/pet 3.jpeg','/content/pet 4.jpeg','/content/pet 5.jpeg']

In [ ]:
test_images = manual_images
test_labels = [1 if 'dog' in fname else 0 for fname in test_images]

Load the Classifier
Use the provided classifier.py to load the CNN architectures.

In [ ]:
# classifier.py example
import torch
from torchvision import models, transforms
from PIL import Image

def load_model(model_name):
    if model_name == "alexnet":
        model = models.alexnet(pretrained=True)
    elif model_name == "vgg":
        model = models.vgg16(pretrained=True)
    elif model_name == "resnet":
        model = models.resnet50(pretrained=True)
    model.eval()
    return model

def classify_image(model, image_path):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img = Image.open(image_path)
    img = preprocess(img)
    img = torch.unsqueeze(img, 0)
    with torch.no_grad():
        output = model(img)
    return output


Load and Preprocess Images

In [ ]:
import os

def preprocess_image(image_path):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img = Image.open(image_path)
    img = preprocess(img)
    img = torch.unsqueeze(img, 0)  # Add batch dimension
    return img


Classify Images

In [ ]:
def classify_image(model, image_path):
    img = preprocess_image(image_path)
    with torch.no_grad():
        output = model(img)
    _, predicted = torch.max(output, 1)
    return predicted.item()  # Return the predicted class index


 Evaluate Classifier Performance

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_classifier(model, test_images, test_labels):
    predictions = [classify_image(model, img_path) for img_path in test_images]
    accuracy = accuracy_score(test_labels, predictions)
    precision = precision_score(test_labels, predictions, average='weighted')
    recall = recall_score(test_labels, predictions, average='weighted')
    f1 = f1_score(test_labels, predictions, average='weighted')
    return accuracy, precision, recall, f1


Time the Classifier

In [ ]:
import time

def time_classifier(model, image_path):
    start_time = time.time()
    classify_image(model, image_path)
    end_time = time.time()
    return end_time - start_time


Compare Multiple Classifiers

In [ ]:
model_names = ["alexnet", "vgg", "resnet"]
results = []

for model_name in model_names:
    model = load_model(model_name)
    accuracy, precision, recall, f1 = evaluate_classifier(model, test_images, test_labels)
    runtimes = [time_classifier(model, img_path) for img_path in test_images]
    avg_runtime = sum(runtimes) / len(runtimes)
    results.append((model_name, accuracy, precision, recall, f1, avg_runtime))

# Choose the best classifier based on the metrics
best_classifier = max(results, key=lambda x: (x[1], -x[5]))  # Prioritize accuracy, then runtime


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344:

In [ ]:
print("Best Classifier:", best_classifier)

Best Classifier: ('alexnet', 0.0, 0.0, 0.0, 0.0, 0.037889623641967775)
